## Plot V11 Shmoo

### 1. Query

In [1]:
from fetch import QueryParam, Query, unwind_tag
from collections import defaultdict

DB_NAME = "redfinch"   # all previously saved data can be pulled by filters (stdf_name and test_text)
EXTRA_PIPELINE = "gt:-100+rssi_min"

query_params = [
    QueryParam(
        stdf_name="sweep_with_svc.stdf.gz",  # stdf should be in the same folder with this notebook 
        text="a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_x_x_rssi",  # _x_ is wildcard
        stdf_id="svc",   # stdf_id is for display only, not saved in data base
    ),
]

df = Query(DB_NAME).get_df(query_params, EXTRA_PIPELINE, [{"$match": {"tag": {"$ne": ""}}}])
df = unwind_tag(df)
df.head()

,rssi_min,count,part_id,site,tag,text,pout,stdf_id,key,v11
0,-82.2500,10,2,2,V11=1.0360,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-82.0,svc,svc_2,1.036
1,-82.4375,10,2,2,V11=0.9930,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-82.0,svc,svc_2,0.993
2,-82.5000,10,2,2,V11=0.9360,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-82.0,svc,svc_2,0.936
3,-82.6250,11,2,2,V11=1.0220,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-82.0,svc,svc_2,1.022
4,-82.3750,10,2,2,V11=1.0510,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-82.0,svc,svc_2,1.051


### 2. Plot V11

In [2]:
from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

data = {}
for column in df.columns:
    data[column] = df[column].tolist()
    
ggplot(data=data) \
    + geom_point(mapping = aes(x='pout', y='rssi_min', color='part_id')) \
    + scale_color_discrete() \
    + facet_wrap(facets="v11",  format=["V11: {} V"], ncol=2)


### 3 Plot SVC Offsets

In [3]:
query_params_svc = [
    QueryParam(
        stdf_name="sweep_with_svc.stdf.gz",  
        text="svc1",  
        stdf_id="svc",
    ),
]

df_svc = Query(DB_NAME).get_df(query_params_svc, "", [{"$project": {"_id": 0, "svc": "$ptr.v", "key": 1}}])
df = df.merge(df_svc, how="left", on="key")
df = df.assign(svc_offset_mv = lambda x: (round((x["v11"] - x["svc"]) / 0.015)) * 15)
df = df[df["svc_offset_mv"] > 0]
df.head()

,rssi_min,count,part_id,site,tag,text,pout,stdf_id,key,v11,svc,svc_offset_mv
0,-82.2500,10,2,2,V11=1.0360,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-82.0,svc,svc_2,1.036,1.022,15.0
4,-82.3750,10,2,2,V11=1.0510,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-82.0,svc,svc_2,1.051,1.022,30.0
11,-82.4375,10,2,2,V11=1.0650,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-82.0,svc,svc_2,1.065,1.022,45.0
12,-76.3750,10,2,2,V11=1.0650,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-76.0,svc,svc_2,1.065,1.022,45.0
17,-76.3125,10,2,2,V11=1.0360,a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20_step...,-76.0,svc,svc_2,1.036,1.022,15.0


In [4]:
ggplot(data=df) \
    + geom_point(mapping = aes(x='pout', y='rssi_min', color='part_id')) \
    + scale_color_discrete() \
    + facet_wrap(facets="svc_offset_mv",  format=["SVC Offset: {} mV"], ncol=2)